<br><font size="5">必要なライブラリのインポート</font></br>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import os
import h5py
from keras.utils import to_categorical
from keras.models import Sequential, load_model, model_from_json,model_from_yaml
from keras.layers import  Input,Dense, Dropout,Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization,Activation
from keras.datasets import mnist,cifar10,cifar100
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras import backend as K

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import scipy
from scipy.stats import norm
import math

<br><font size="5">Keras公式のVGG16(ImageNet)をダウンロード</font><br>
<font size="5" color="red">※本実験では用いない</font>

In [ ]:
model=keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
model.summary()

<br><font size="5">データセット：MNISTの場合</font><br>
<font size="5" color="red">※本実験では用いない</font>

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 28 x 28の画像がgrayscaleで1chなので、28, 28, 1にreshapeする
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

#0-255の整数値を0〜1の小数に変換する
#MNISTって必ずこの処理入るけれど、意味あるのかな
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one-hot vector形式に変換する
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

<br><font size="5">データセット：Cifar-10の場合</font>

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one-hot vector形式に変換する
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


<br><font size="5">データセット：Cifar-100の場合</font>

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one-hot vector形式に変換する
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


<br><font size="5">ネットワーク：VGG-16<br>
データセット：MNIST,Cifar-10,Cifar-100の場合<br>
重みの初期値：Xavier<br>
クラス分類</font>

In [ ]:
model = Sequential()

#MNISTの場合
#model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name="Conv1",input_shape=(28,28,1)))
#cifar10,cifar100の場合
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name="Conv1",input_shape=(32, 32, 3)))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name="Conv2"))
model.add(BatchNormalization(name="Norm1"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool1"))

model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv3"))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1),activation='relu', padding='same',name="Conv4"))
model.add(BatchNormalization(name="Norm2"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool2"))

model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv5"))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv6"))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv7"))
model.add(BatchNormalization(name="Norm3"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool3"))

model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv8"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv9"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv10"))
model.add(BatchNormalization(name="Norm4"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool4"))

model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv11"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv12"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',name="Conv13"))
model.add(BatchNormalization(name="Norm5"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool5"))

model.add(Flatten(name="Flat"))

model.add(Dense(units=512,activation="relu",name="Dense1"))
model.add(BatchNormalization(name="Norm6"))

model.add(Dense(units=512,activation="relu",name="Dense2"))
model.add(BatchNormalization(name="Norm7"))

#MNIST,cifar10の場合
#model.add(Dense(units=10, activation='softmax',name='Dense3'))
#cifar100の場合
model.add(Dense(units=100, activation='softmax',name='Dense3'))

model.summary()

<br><font size="5">ネットワーク：VGG-16<br>
データセット：MNIST,Cifar-10,Cifar-100の場合<br>
重みの初期値：He_normal(+正則化L2)<br>
クラス分類</font>

In [ ]:
from keras.regularizers import l2

model = Sequential()

#MNISTの場合
#model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name="Conv1",input_shape=(28,28,1)))
#cifar10,cifar100の場合
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same', name="Conv1",input_shape=(32, 32, 3)))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same', name="Conv2"))
model.add(BatchNormalization(name="Norm1"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool1"))

model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu',kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv3"))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1),activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv4"))
model.add(BatchNormalization(name="Norm2"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool2"))

model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv5"))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv6"))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv7"))
model.add(BatchNormalization(name="Norm3"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool3"))

model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv8"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv9"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv10"))
model.add(BatchNormalization(name="Norm4"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool4"))

model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv11"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv12"))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),padding='same',name="Conv13"))
model.add(BatchNormalization(name="Norm5"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same',name="Pool5"))

model.add(Flatten(name="Flat"))
model.add(Dense(units=512,activation='relu',kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),name="Dense1"))
model.add(BatchNormalization(name="Norm6"))
#model.add(Activation('relu',name="Activation1"))

model.add(Dense(units=512,activation='relu',kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),name="Dense2"))
model.add(BatchNormalization(name="Norm7"))
#model.add(Activation('relu',name="Activation2"))

#MNIST,cifar10の場合
#model.add(Dense(units=10, activation='softmax', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),name='Dense3'))
#cifar100の場合
model.add(Dense(units=100, activation='softmax', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4),name='Dense3'))

model.summary()

<br><font size="5">モデルコンパイル</font>

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

'''
ここで，学習率減衰したかったが，実装できなかった
'''

# def lr_schedule(epoch):
#     lrate=0.001
#     if epoch>=100:
#         lrate=0.0002
#     elif epoch>=150:
#         lrate=0.00004
#     return lrate


# scheduler = LearningRateScheduler(schedule=lr_schedule)

adam = Adam(lr=0.001) # 最適化手法(Adam)

model.compile(loss='categorical_crossentropy',
              optimizer=adam, 
              metrics=['accuracy'])

SyntaxError: EOL while scanning string literal (<ipython-input-3-44f9b1efee92>, line 8)

<br><font size="5">モデルの学習</font>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#データオーグメンテーション
datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    channel_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    #vertical_flip=True
)

# 訓練用パラメータの設定.
batch_size = 128
epochs = 200

history = model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size, 
    validation_data=(x_test, y_test),
    epochs=epochs, 
    verbose=1,)

<br><font size="5">モデルの精度と損失のグラフをプロット</font>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['accuracy'],"-",label="accuracy")
    plt.plot(history.history['val_accuracy'],"-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"-",label="loss",)
    plt.plot(history.history['val_loss'],"-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    plt.show()
# modelに学習させた時の変化の様子をplot
plot_history(history)

<br><font size="5">モデルの精度のグラフを保存</font>

In [ ]:
#ファイル名を決める
f_model='./VGG16_cifar100_epoch200_he_normal'

matplotlib.use('Agg') 
plt.plot(history.history["val_accuracy"],marker="x",label="val_accuracy")
plt.plot(history.history["accuracy"],marker=".",label="accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend(loc="best")
plt.savefig(os.path.join(f_model, "認識率_accuracy"+".tif"))
plt.show()

<br><font size="5">モデルの損失のグラフを保存</font>

In [ ]:
plt.plot(history.history["val_loss"],marker="x",label="val_loss") 
plt.plot(history.history["loss"],marker=".",label="loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(loc="best")
plt.savefig(os.path.join(f_model, "認識率_loss"+".tif"))
plt.show()

<br><font size="5">モデルの重み係数を保存</font>

In [ ]:
# f_model="VGG16_"   #""(ダブルコーテーション)又は''(シングルコーテーション)
# step1="cifar100_epoch200_vector2_class256"      #""(ダブルコーテーション)又は''(シングルコーテーション)   

#f_model2="VGG16_cifar100_epoch200_vector2_class128"
#f_model2="./VGG16_he_normal/cifar10/vector2_1_8_400_9_13_10(1)"
f_model2="./VGG16_he_normal/cifar100/vector3/class10(1)"
#f_model2="./VGG16/cifar100/vector4/class100(1)"
#f_model2=f_model+"_step"+str(step1)

os.mkdir(f_model2)
print('save the architecture of a model')
json_string=model.to_json()                                           #モデル構造の保存(9行目まで)
open(os.path.join(f_model2,'cnn_model.json'),'w').write(json_string)
yaml_string=model.to_yaml()
open(os.path.join(f_model2,'cnn_model.yaml'),'w').write(yaml_string)
print('save weights')
model.save_weights(os.path.join(f_model2,'cnn_model_weights.hdf5'))  #重み係数の保存
print('save all files')

save the architecture of a model
save weights
save all files


<br><font size="5">学習済みモデルの読み込み</font>

In [ ]:
f_model = './VGG16_cifar10_epoch200_he_normal'
#f_model = './VGG16_he_normal/cifar10/vector2/class15(1)'
#f_model='./処理後/VGG16_cifar10_epoch200_3_(1)512_(2_11)256_(12_13)16'
model_filename = 'cnn_model.json'
weights_filename = 'cnn_model_weights.hdf5'

json_string = open(os.path.join(f_model, model_filename)).read()
model = model_from_json(json_string)
model.load_weights(os.path.join(f_model,weights_filename)) #学習済みの重みを利用
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 32, 32, 64)        1792      
_________________________________________________________________
Conv2 (Conv2D)               (None, 32, 32, 64)        36928     
_________________________________________________________________
Norm1 (BatchNormalization)   (None, 32, 32, 64)        256       
_________________________________________________________________
Pool1 (MaxPooling2D)         (None, 16, 16, 64)        0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 16, 16, 128)       73856     
_________________________________________________________________
Conv4 (Conv2D)               (None, 16, 16, 128)       147584    
_________________________________________________________________
Norm2 (BatchNormalization)   (None, 16, 16, 128)      

<br><font size="5">モデルコンパイル</font>

In [ ]:
from keras.optimizers import Adam

adam = Adam(lr=0.001) # 最適化手法(Adam)

model.compile(loss='categorical_crossentropy',
              optimizer=adam, 
              metrics=['accuracy'])

<br><font size="5">学習データと訓練データの精度と損失値</font>

In [ ]:
score = model.evaluate(x_train,y_train, batch_size=128) #データセット
print("loss = {:.5f}".format(score[0]))
print("accuracy = {:.5f}".format(score[1]))
score = model.evaluate(x_test,y_test, batch_size=128) #訓練データ
print("loss = {:.5f}".format(score[0]))
print("accuracy = {:.5f}".format(score[1]))

50000/50000 [==============================] - 5s 105us/step
loss = 1.59843
accuracy = 0.76464
10000/10000 [==============================] - 1s 103us/step
loss = 2.30055
accuracy = 0.61930
time:6.5620129108428955[sec]



<br><font size="5">訓練データの精度と損失値</font>

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

<br><font size="5">量子化後の認識率の比較</font>

In [ ]:
from PIL import Image
import sys


pre = model.predict(x_test)
count=0
for i,v in enumerate(pre):
    pre_ans = v.argmax()
    ans = y_test[i].argmax()
    dat = x_test[i]
    if ans == pre_ans:
        count=count+1
        continue
result=count/10000  #訓練画像の枚数

data1=['val_acc',result]
result=result*100
result=np.round(result,1)
data2=['result(%)',result]

print(result)

1.0


<br><font size="5">ベクトル量子化</font>

In [ ]:
#np.whereでラベルを各クラスタの重心に置き換えて，それを重み係数に代入して保存する．
#畳み込み層第1層目から13層目まで各層の重み係数をクラスタリング(kmeans++)してる．

import csv
from sklearn.cluster import KMeans

#filename="VGG16_cifar10_epoch200"

#各変数の初期化
entropy_sum=0
size_sum=0
bit_all=0
entropy_all=0

#各層ごとにコードブック(代表ベクトルを集めたデータ群)を作成して，ベクトル量子化を行う関数
def Vector_Quantization(w,b,shape,num,num_2):
    
    w=w.reshape(int(w.size/num),-1)
#     print("------------------reshape後----------------------------------------------------")
#     print(w.shape)
#     print(w)
#     print("---------------------------------------------------------------------------------")
    
    # K-means++ クラスタリングをおこなう
    kmeans_model = KMeans(n_clusters=num_2).fit(w)
    # 分類先となったラベルとクラスターを取得する
    labels = kmeans_model.labels_
    clusters=kmeans_model.cluster_centers_
    
    #print(labels)
    
    #ラベルとクラスターをリスト型に変換する
    labels_2=labels.tolist()
    clusters_2=clusters.tolist()
    
    #print(labels_2)
    
    #ここで各コードブックの発生回数を求めている，binsはクラスター数を入れている
    histogram,_=np.histogram(labels_2,bins=num_2)
    #print(histogram)
    #ここで平均情報量を求めている,baseはlogの底を表している
    entropy=scipy.stats.entropy(histogram,base=2)
    
    #エントロピー算出の際，ベクトル量子化だと次元数で割る
    vq_entropy=(entropy/num)*w.size
    
#     print("------------------ベクトル量子化後のエントロピー----------------------------------------------------")
#     print(vq_entropy)
#     print("-------------------------------------------------------------------------------------------------")
    
    #ラベルの数を増やすため，insert(挿入)する位置をここで生成している
    square=[0 for a in range(num-1)]
    for j in range(1,num):
        square[j-1]=[i for i in range(0,int(w.size/num*j),j)]
    #実際に同じ値のラベルを挿入してreshapeできるようにしている
    data=[0 for a in range(num)]
    data[0]=labels    
    for j in range(1,num):
        data[j]=np.insert(data[j-1],square[j-1],labels_2)
    #ここでラベル配列(一次元)を二次元配列にreshapeする
    box=data[-1].reshape(int(w.size/num),-1)
    
#     print("------------------ラベル配列の様子----------------------------------------------------")
#     print(box)
#     print("-------------------------------------------------------------------------------------")
    
#     print("------------------クラスターの重心----------------------------------------------------")
#     print(clusters)
#     print("-------------------------------------------------------------------------------------")
    
    #全ラベル[次元数の数]をcsvファイルに
#     with open(filename+"_box.csv", "a") as f:
#         #writer = csv.writer(f, lineterminator='\n') # 行末は改行
#         writer = csv.writer(f) 
#         #writer.writerow(b)
#         for d in box:
#             writer.writerow(d)
        
    #全クラスターをcsvファイルに
#     with open(filename+"_clusters.csv", "a") as f:
#         #writer = csv.writer(f, lineterminator='\n') # 行末は改行
#         writer = csv.writer(f)
#         #writer.writerow(clusters)
#         for c in clusters:
#             writer.writerow(c)
        
    #ここでラベルだけの配列に各クラスターの重心を代入する
    data_array = [0 for b in range(num_2)]
    data_array[0] =np.where(box==[0]*num,clusters_2[0],box)
    for k in range(1,num_2):
        data_array[k] = np.where(data_array[k-1]==[k for jj in range(num)],clusters_2[k],data_array[k-1])
    #ここでVQ後の重みを代入する
    w=data_array[-1]
    
#          #全クラスターをcsvファイルに
#     with open(filename+"_weights.csv", "a") as f:
#         #writer = csv.writer(f, lineterminator='\n') # 行末は改行
#         writer = csv.writer(f)
#         #writer.writerow(clusters)
#         writer.writerow(w)
    
    """}
    array_1=np.where(data_3==[0,0,0],clusters_2[0],data_3)
    array_2=np.where(array_1==[1,1,1],clusters_2[1],array_1)
    array_3=np.where(array_2==[2,2,2],clusters_2[2],array_2)
    array_4=np.where(array_3==[3,3,3],clusters_2[3],array_3)
    array_5=np.where(array_4==[4,4,4],clusters_2[4],array_4)
    array_6=np.where(array_5==[5,5,5],clusters_2[5],array_5)
    array_7=np.where(array_6==[6,6,6],clusters_2[6],array_6)
    array_8=np.where(array_7==[7,7,7],clusters_2[7],array_7)
    array_9=np.where(array_8==[8,8,8],clusters_2[8],array_8)
    
    w=array_9
    """
    #ここで元の重みの型にreshapeする
    w=w.reshape(shape)
    
    #ここで重みを保存
    model.get_layer("Conv"+str(i)).set_weights([w,b])
    
#     print("------------------ベクトル量子化後の重み----------------------------------------------------")
#     print(w)
#     print("------------------------------------------------------------------------------------------")
    
    #VQ後のエントロピーと重み係数の個数を返り値としている
    return vq_entropy,w.size

    
#畳み込み層(1~13層目)
for i in range(1,14):
    w=model.get_layer("Conv"+str(i)).get_weights()[0] #重み
    b=model.get_layer("Conv"+str(i)).get_weights()[1] #バイアス
    shape=w.shape
    print("Conv"+str(i))
#     print("------------------"+"Conv"+str(i)+"の重み"+"----------------------------------------------------")
#     print(shape)
#     print(w)
#     print("------------------------------------------------------------------------------------------------")
    #(重み，バイアス，重みのshape，入力次元数，分けるクラス数)
    entropy,size=Vector_Quantization(w,b,shape,3,10)
    #エントロピーと重み係数の個数を足している
    entropy_sum+=entropy
    size_sum+=size
#     print("------------------"+"Conv"+str(i)+"までのエントロピー"+"----------------------------------------------------")
#     print(entropy_sum / size_sum )
#     print("--------------------------------------------------------------------------------------------------------------")
    
#ここで全体のエントロピー算出
print(entropy_sum / size_sum )

Conv1
------------------Conv1の重み----------------------------------------------------
(3, 3, 3, 64)
------------------------------------------------------------------------------------------------
Conv2
------------------Conv2の重み----------------------------------------------------
(3, 3, 64, 64)
------------------------------------------------------------------------------------------------
Conv3
------------------Conv3の重み----------------------------------------------------
(3, 3, 64, 128)
------------------------------------------------------------------------------------------------
Conv4
------------------Conv4の重み----------------------------------------------------
(3, 3, 128, 128)
------------------------------------------------------------------------------------------------
Conv5
------------------Conv5の重み----------------------------------------------------
(3, 3, 128, 256)
------------------------------------------------------------------------------------------------
Conv6
-----

In [ ]:
#第1層目の重み係数を取り出す場合
i=1
weights=model.get_layer("Conv"+str(i)).get_weights()[0]
weights

array([[[[-2.91950273e-32, -1.32626957e-32, -1.65118892e-02, ...,
           2.94441293e-32, -3.08294058e-01, -2.72437302e-32],
         [-2.89190800e-32, -2.26068091e-32, -2.23156258e-01, ...,
          -2.86312455e-32, -3.97022754e-01, -2.78202926e-32],
         [ 2.94850306e-32,  2.44034097e-32, -1.66881815e-01, ...,
          -3.02844755e-32, -5.85686862e-01, -2.52653468e-32]],

        [[-2.58212028e-32, -6.38503111e-33,  4.15137589e-01, ...,
          -2.81060317e-32,  2.59104729e-01, -2.75944801e-32],
         [-2.71475101e-32, -2.42194183e-32,  3.90448928e-01, ...,
          -2.93331185e-32,  4.70458925e-01, -2.95500766e-32],
         [-2.74026394e-32, -2.59474744e-32,  8.45992088e-01, ...,
           2.85805053e-32,  1.41120091e-01, -2.68767526e-32]],

        [[ 3.00161483e-32, -9.76388530e-33, -5.63182771e-01, ...,
          -2.74471760e-32,  1.06783256e-01,  2.73184975e-32],
         [ 2.83399169e-32, -2.38070153e-32, -1.46186441e-01, ...,
          -2.73738045e-32,  3.8400

<br><font size="5">重み係数(畳み込み層1～13層目)の統計的性質をcsvファイルにまとめる</font>

In [ ]:
import csv

#csvファイルのファイル名を決める
filename="VGG16_cifar100_epoch200_he_normal"

# #1層目の重み係数の統計的性質をそのまま出力する場合
# weights=model.get_layer("conv2d").get_weights()[0]
# weights=weights.reshape(-1)
# max=np.amax(weights) #重み係数の最大値
# min=np.amin(weights) #重み係数の最小値
# avr=np.mean(weights) #重み係数の平均値
# dis=np.var(weights) #重み係数の分散
# hensa=np.std(weights) #重み係数の分散
# kurtosis = scipy.stats.kurtosis(weights)  #重み係数の尖度
# skew = scipy.stats.skew(weights)  #重み係数の歪度
# count_2=len(weights)  #重み係数の数
# print("畳み込み層　1層目")
# print("最大値: "+str(max)+"　最小値: "+str(min)+"　平均値: "+str(avr)+"　分散: "+str(dis)+"　標準偏差: "+str(hensa)+"　歪度: "+str(skew)+"　尖度: "+str(kurtosis)+"　係数の数: "+str(count_2))

#各層の統計的性質をcsvファイルにまとめる関数(ただ記述しているだけ)
def weights_text(i,max,min,avr,dis,hensa,skew,kutosis,count_2): 
    name=['畳み込み層第'+str(i)+'層目',max,min,avr,hensa,skew,kurtosis]
    data1=['最大値',max]
    data2=['最小値',min]
    data3=['平均値',avr]
    data4=['分散',dis]
    data5=['標準偏差',hensa]
    data6=['歪度',skew]
    data7=['尖度',kurtosis]
    data8=['係数の数',count_2]

    #with open(filename+"weights_hist_step"+str(step1)+".csv", "a") as f:
    with open(filename+".csv", "a") as f:
        writer = csv.writer(f, lineterminator='\n') # 行末は改行
        writer.writerow(name) 
        
#         writer.writerow(data1)
#         writer.writerow(data2)
#         writer.writerow(data3)
#         writer.writerow(data4)
#         writer.writerow(data5)
#         writer.writerow(data6)
#         writer.writerow(data7)
#         writer.writerow(data8)

#各層の統計的性質をcsvファイルに出力するための下準備的な関数
def Layer(weights): 
    
    #変数の初期化
    count=0
    max=0
    min=0
    avr=0
    dis=0
    bunsan=0
    skew=0
    kurtosis=0
    count_2=0

    for w in weights[:]:
        count=count+1
    max=np.amax(weights) #重み係数の最大値
    min=np.amin(weights) #重み係数の最小値
    avr=np.mean(weights) #重み係数の平均値
    dis=np.var(weights) #重み係数の分散
    hensa=np.std(weights) #重み係数の標準偏差
    kurtosis = scipy.stats.kurtosis(weights)  #重み係数の尖度
    skew = scipy.stats.skew(weights)  #重み係数の歪度
    count_2=len(weights)   #重み係数の個数
    #print(max,min,avr,hensa)
    #print("weights_count: "+str(count))
    
    #ここで各統計的性質を返り値としている
    return max,min,avr,dis,hensa,kurtosis,skew,count_2

#畳み込み層(1～13層)
for i in range(1,14):
    weights=model.get_layer("Conv"+str(i)).get_weights()[0]
    weights=weights.reshape(-1)
    
    max,min,avr,dis,hensa,kurtosis,skew,count_2=Layer(weights)
    print("畳み込み層　"+str(i)+"層目")
    print("最大値: "+str(max)+"　最小値: "+str(min)+"　平均値: "+str(avr)+"　分散: "+str(dis)+"　標準偏差: "+str(hensa)+"　歪度: "+str(skew)+"　尖度: "+str(kurtosis)+"　係数の数: "+str(count_2))
    weights_text(i,max,min,avr,dis,hensa,skew,kurtosis,count_2)

畳み込み層　1層目
最大値: 0.9078934　最小値: -0.87226695　平均値: -0.006415184　分散: 0.05793824　標準偏差: 0.24070364　歪度: 0.08025909960269928　尖度: 1.886154609142391　係数の数: 1728
畳み込み層　2層目
最大値: 0.74438274　最小値: -1.0613595　平均値: -0.0054214173　分散: 0.006776381　標準偏差: 0.082318775　歪度: -0.47914958000183105　尖度: 9.949713237478058　係数の数: 36864
畳み込み層　3層目
最大値: 0.6735768　最小値: -0.7685852　平均値: -0.0043728845　分散: 0.0069083595　標準偏差: 0.08311654　歪度: -0.3882627487182617　尖度: 3.878866248751665　係数の数: 73728
畳み込み層　4層目
最大値: 0.7277174　最小値: -0.55779904　平均値: -0.0087076835　分散: 0.005726605　標準偏差: 0.07567433　歪度: 0.2297566831111908　尖度: 1.253432417289229　係数の数: 147456
畳み込み層　5層目
最大値: 0.62851304　最小値: -0.4033634　平均値: -0.0096622575　分散: 0.0053816284　標準偏差: 0.07335959　歪度: 0.21293330192565918　尖度: 0.8723012938243002　係数の数: 294912
畳み込み層　6層目
最大値: 0.54739255　最小値: -0.31805608　平均値: -0.0055891224　分散: 0.0027864592　標準偏差: 0.052786924　歪度: 0.39123547077178955　尖度: 3.5100582465190353　係数の数: 589824
畳み込み層　7層目
最大値: 0.5718624　最小値: -0.27715802　平均値: -0.0038936755　分散: 0.0019084164　標準偏

<br><font size="5">重み係数(畳み込み層1～13層目)の分布をそれぞれtifファイルに出力</font>

In [ ]:

#ファイル名を決める
f_model = './VGG16_cifar100_epoch200_he_normal/weight_1.0_bins100'

def weights_hist(weights_array,i): 
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    #x軸(重み係数)
    ax.set_xlabel('weight',fontsize=12)
    #y軸(発生確率)
    ax.set_ylabel('probability(%)',fontsize=12)
    
    #ax.set_xticks([-0.08, -0.04, 0, 0.04, 0.08]) 
    #ax.set_yticks([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30])
    #ax.set_yticks([0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52])
    plt.title("畳み込み層　"+str(i)+"層目",fontname="MS Gothic")
    plt.tick_params(labelsize=12)
    #ax.set_xticks([-0.08, -0.04, 0, 0.04, 0.08]) 
    #ax.set_yticks([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30])
    #ax.set_yticks([0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52])
    #plt.tick_params(labelsize=12)
    plt.hist(weights_array,range=(-1.0,1.0),bins=100,density=True)
    #plt.hist(weights_array, range=(-0.11,0.11), density=True, histtype='barstacked', ec='black')
    #ax.hist(weights_array)
    plt.savefig(os.path.join(f_model, "畳み込み層"+str(i)+"層目_weight"+".tif")) 

#畳み込み層(1～13層)
for i in range(1,14):
    weights=model.get_layer("Conv"+str(i)).get_weights()[0]
    weights=weights.reshape(-1)
    weights_hist(weights,i)    